Data:

For this project I will be using two primary sources of data.
1) location data of Houston Neighborhoods. 
2) Foursquare venue data

Location Data: 
I have put togehter a google sheet table of the 14 Houston neighborhoods I will be targeting along with their respective latitudes and longitueds.This data will be used to feed the Foursquare API in order to find restaurants and venues in that neighborhood. 

Foursquare venue data:
By feeding the houston neighborhood data into the foursquare API, I can generate a list of the top 10 venues in each neighborhood. 
With this information I will be able to select the neighborhood that has the greatest proportion of restuarants in its area, but with very few existing pizza restaurants.  
This will allow be to objectively choose the best location to place my new pizza restaurant. 

In [2]:
#Location Data:
import pandas as pd
!wget -q -O 'h_data.csv' https://docs.google.com/spreadsheets/d/1HviOJilytLsUzM27PtmuYx3I6lQDKcPMx9MfioRkk9k/gviz/tq?tqx=out:csv
neighborhoods = pd.read_csv("h_data.csv")
neighborhoods

,Neighborhood,Latitude,Longitude
0,Garden Oaks / Oak Forest,29.826345,-95.417523
1,The Heights / Timbergrove,29.796311,-95.413312
2,Memorial,29.770561,-95.508688
3,Uptown / Galleria,29.740464,-95.479825
4,Memorial Park / Rice Military,29.770999,-95.406581
5,River Oaks,29.750225,-95.426855
6,Highland Village,29.736313,-95.448954
7,Shepherd Kirby,29.736508,-95.413534
8,Montrose,29.752623,-95.384910
9,Downtown / Midtown,29.756378,-95.363284


In [4]:
#Foursquare Data:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas datafram
#Define 4Square Credentials
CLIENT_ID = 'Y3CRNKK4MYSH0MUSM05PCWBBXJQ4TBBHXIGSFJSC15OJS13U' # your Foursquare ID
CLIENT_SECRET = 'QEGJVIW1HUVLTHVWRVF4AO5FVTXFP5VSFLNEZ2I5MSZ32Y0A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    #Function to get top venues from each Target Neighborhood in Houston
#Set radius at 2,000m as average neighborhood has a diameter of about 4,000 meters
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

houston_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
houston_venues.head()

Garden Oaks / Oak Forest
The Heights / Timbergrove
Memorial
Uptown / Galleria
Memorial Park / Rice Military
River Oaks
Highland Village
Shepherd Kirby
Montrose
Downtown / Midtown
Museum District
West University
Medical Center
Bellaire


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Garden Oaks / Oak Forest,29.826345,-95.417523,Bernies Backyard,29.826904,-95.410065,Food Truck
1,Garden Oaks / Oak Forest,29.826345,-95.417523,Brothers Pizzeria,29.821651,-95.409890,Pizza Place
2,Garden Oaks / Oak Forest,29.826345,-95.417523,LA Fitness,29.828649,-95.409937,Gym
3,Garden Oaks / Oak Forest,29.826345,-95.417523,Petrol Station,29.821395,-95.424424,Bar
4,Garden Oaks / Oak Forest,29.826345,-95.417523,Cottonwood,29.818285,-95.410011,Beer Garden
